In [1]:
import pandas as pd
import numpy as np

In [30]:
df = pd.read_csv('data/roads.csv')

def genCmd(points):
    cmd = []
    for p in points:
        x, y = p
        cmd.append('new google.maps.LatLng(%f, %f),' % (x, y))
    
    return "new google.maps.Polyline({path:[%s], geodesic: true, strokeColor: '#11FF00',\
        strokeOpacity: 1.0, strokeWeight: 2, map:map});" % ''.join(cmd)

def extractTrack(idx):
#     print df['Track'][idx]
    points = []
    for x in df['Track'][idx].split(';'):
        points.append([float(t) for t in x.split(',')[::-1]])
    return points
        


points = extractTrack(2)
q = genCmd(points)
    
    
    


In [31]:
full = []
for i,_ in enumerate(df['Track']):
    points = extractTrack(i)
    q = genCmd(points)
    full.append(q)
with open('out.txt', 'w') as fout:
    fout.writelines(full)